<a href="https://colab.research.google.com/github/anshupandey/AI_Agents/blob/main/AAP_UC3_code_retrieval_augmented_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use Retrieval Augmented Generation (RAG) with Codey APIs

### Objective

This notebook demonstrates how you augment output from Codey APIs by bringing in external knowledge. We'll show you an example using Code Retrieval Augmented Generation(RAG) pattern using [Google Cloud's Generative AI github repository](https://github.com/GoogleCloudPlatform/generative-ai) as external knowledge.The notebook uses [Vertex AI PaLM API for Code](https://cloud.google.com/vertex-ai/docs/generative-ai/code/code-models-overview), [Embeddings for Text API](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings), FAISS vector store and [LangChain 🦜️🔗](https://python.langchain.com/en/latest/).

### Overview

Here is overview of what we'll go over.

Index Creation:

1. Recursively list the files(.ipynb) in github repo
2. Extract code and markdown from the files
3. Chunk & generate embeddings for each code strings and add initialize the vector store

Runtime:

4. User enters a prompt or asks a question as a prompt
5. Try zero-shot prompt
6. Run prompt using RAG Chain & compare results.To generate response we use **code-bison** however can also use **code-gecko** and **codechat-bison**

### Cost

This tutorial uses billable components of Google Cloud:

- Vertex AI PaLM APIs offered by Google Cloud

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

**Note:** We are using local vector store(FAISS) for this example however recommend managed highly scalable vector store for production usage such as [Vertex AI Matching Engine](https://cloud.google.com/vertex-ai/docs/vector-search/overview) or [AlloyDB for PostgreSQL](https://cloud.google.com/alloydb/docs/ai/work-with-embeddings) or [Cloud SQL for PostgreSQL](https://cloud.google.com/sql/docs/postgres/features)  using pgvector extension.

### Install libraries

In [2]:
!pip install --upgrade --user -q google-cloud-aiplatform langchain faiss-cpu
!pip install --user --quiet langchain langchain-chroma langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.1 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3

### Restart runtime

In [3]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, you will need to authenticate your environment. To do this, run the cell below. This step is not required if you are using Vertex AI Workbench.

In [1]:
import sys

if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Import libraries

In [2]:
from typing import List
import nbformat
import requests
import time


from langchain.schema.document import Document

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import Language
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Vertex AI
from google.cloud import aiplatform
import vertexai

print(f"Vertex AI SDK version: {aiplatform.__version__}")

Vertex AI SDK version: 1.60.0


In [6]:
!pip install langchain-google-vertexai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 5.9 MB/s eta 0:00:00


In [9]:
# Initialize project
# Define project information
PROJECT_ID = "jrproject-402905"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

vertexai.init(project=PROJECT_ID, location=LOCATION)

from langchain_google_vertexai import VertexAI
# Code Generation
code_llm = VertexAI(
    model_name="code-bison@002",
    max_output_tokens=2048,
    temperature=0.1,
    verbose=False,
)

Next we need to create a GitHub personal token to be able to list all files in a repository.

- Follow [this link](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens) to create GitHub token with repo->public_repo scope and update `GITHUB_TOKEN` variable below.

In [10]:
# provide GitHub personal access token
GITHUB_TOKEN = "xxxxxxxxxxxxxxxxxxxxxx"  # @param {type:"string"}
GITHUB_REPO = "GoogleCloudPlatform/generative-ai"  # @param {type:"string"}

# Index Creation

We will be using the Google Cloud Generative AI github repository as the data source. First list all Jupyter Notebook files in the repo and store it in a text file.

You can skip this step(#1) if you have executed it once and generated the output text file.

### 1. Recursively list the files(.ipynb) in the github repository

In [11]:
# Crawls a GitHub repository and returns a list of all ipynb files in the repository
def crawl_github_repo(url: str, is_sub_dir: bool, access_token: str = GITHUB_TOKEN):
    ignore_list = ["__init__.py"]

    if not is_sub_dir:
        api_url = f"https://api.github.com/repos/{url}/contents"

    else:
        api_url = url

    headers = {
        "Accept": "application/vnd.github.v3+json",
        "Authorization": f"Bearer {access_token}",
    }

    response = requests.get(api_url, headers=headers)
    response.raise_for_status()  # Check for any request errors

    files = []

    contents = response.json()

    for item in contents:
        if (
            item["type"] == "file"
            and item["name"] not in ignore_list
            and (item["name"].endswith(".py") or item["name"].endswith(".ipynb"))
        ):
            files.append(item["html_url"])
        elif item["type"] == "dir" and not item["name"].startswith("."):
            sub_files = crawl_github_repo(item["url"], True)
            time.sleep(0.1)
            files.extend(sub_files)

    return files

In [12]:
code_files_urls = crawl_github_repo(GITHUB_REPO, False, GITHUB_TOKEN)

# Write list to a file so you do not have to download each time
with open("code_files_urls.txt", "w") as f:
    for item in code_files_urls:
        f.write(item + "\n")

len(code_files_urls)

206

In [13]:
code_files_urls[0:10]

['https://github.com/GoogleCloudPlatform/generative-ai/blob/main/conversation/data-store-status-checker/data_store_checker.ipynb',
 'https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/embedding-similarity-visualization.ipynb',
 'https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/intro-textemb-vectorsearch.ipynb',
 'https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/intro_embeddings_tuning.ipynb',
 'https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/intro_multimodal_embeddings.ipynb',
 'https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/use-cases/outlier-detection/bq-vector-search-log-outlier-detection.ipynb',
 'https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/vector-search-quickstart.ipynb',
 'https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/chat-completions/intro_chat_completions_api.ipynb',
 'https://github.com/GoogleCloudPla

### 2. Extract code from the Jupyter notebooks.

You could also include .py file, shell scripts etc.

In [14]:
# Extracts the python code from an ipynb file from github
def extract_python_code_from_ipynb(github_url, cell_type="code"):
    raw_url = github_url.replace("github.com", "raw.githubusercontent.com").replace(
        "/blob/", "/"
    )

    response = requests.get(raw_url)
    response.raise_for_status()  # Check for any request errors

    notebook_content = response.text

    notebook = nbformat.reads(notebook_content, as_version=nbformat.NO_CONVERT)

    python_code = None

    for cell in notebook.cells:
        if cell.cell_type == cell_type:
            if not python_code:
                python_code = cell.source
            else:
                python_code += "\n" + cell.source

    return python_code


def extract_python_code_from_py(github_url):
    raw_url = github_url.replace("github.com", "raw.githubusercontent.com").replace(
        "/blob/", "/"
    )

    response = requests.get(raw_url)
    response.raise_for_status()  # Check for any request errors

    python_code = response.text

    return python_code

In [15]:
with open("code_files_urls.txt") as f:
    code_files_urls = f.read().splitlines()
len(code_files_urls)

206

In [16]:
code_strings = []

for i in range(0, len(code_files_urls)):
    if code_files_urls[i].endswith(".ipynb"):
        content = extract_python_code_from_ipynb(code_files_urls[i], "code")
        doc = Document(
            page_content=content, metadata={"url": code_files_urls[i], "file_index": i}
        )
        code_strings.append(doc)

/usr/local/lib/python3.10/dist-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


In [ ]:
print(code_strings[0].page_content)

In [18]:
len(code_strings)

151

### 3. Chunk & generate embeddings for each code strings & initialize the vector store

We need to split code into usable chunks that the LLM can use for code generation. Therefore it's crucial to use the right chunking approach and chunk size.

In [24]:
from langchain_google_vertexai.embeddings import VertexAIEmbeddings

In [25]:
# Utility functions for Embeddings API with rate limiting
def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)


class CustomVertexAIEmbeddings(VertexAIEmbeddings):
    requests_per_minute: int
    num_instances_per_batch: int

    # Overriding embed_documents method
    def embed_documents(self, texts: List[str]):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        docs = list(texts)

        while docs:
            # Working in batches because the API accepts maximum 5
            # documents per request to get embeddings
            head, docs = (
                docs[: self.num_instances_per_batch],
                docs[self.num_instances_per_batch :],
            )
            chunk = self.client.get_embeddings(head)
            results.extend(chunk)
            next(limiter)

        return [r.values for r in results]

In [26]:
# Chunk code strings
text_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=3000, chunk_overlap=400
)


texts = text_splitter.split_documents(code_strings)
print(len(texts))

709


In [27]:
# Initialize Embedding API
EMBEDDING_QPM = 100
EMBEDDING_NUM_BATCH = 5
embeddings = CustomVertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
    model_name="textembedding-gecko@latest",
)

# Create Index from embedded code chunks
db = FAISS.from_documents(texts, embeddings)

# Init your retriever.
retriever = db.as_retriever(
    search_type="similarity",  # Also test "similarity", "mmr"
    search_kwargs={"k": 5},
)

retriever

Waiting
.............................................................................................................................................

VectorStoreRetriever(tags=['FAISS', 'CustomVertexAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x78406650f430>, search_kwargs={'k': 5})

In [29]:
query = "implementing langchain agent with gemini-1.5-flash model"
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content)

# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
! pip3 install --user install ragas==0.1.6 \
datasets==2.18.0 \
langchain==0.1.14 \
langchain-google-vertexai==1.0.5 \
langchain-chroma==0.1.1 \
chromadb==0.5.0 \
pypdf==4.2.0 \
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)
import pandas as pd
import vertexai

from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain.chains import Re

# Runtime
### 4. User enters a prompt or asks a question as a prompt

In [30]:
user_question = "Create a Python function that takes a prompt and predicts using langchain.llms interface with Vertex AI text-bison model"

In [31]:
# Define prompt templates


# Zero Shot prompt template
prompt_zero_shot = """
    You are a proficient python developer. Respond with the syntactically correct & concise code for to the question below.

    Question:
    {question}

    Output Code :
    """

prompt_prompt_zero_shot = PromptTemplate(
    input_variables=["question"],
    template=prompt_zero_shot,
)


# RAG template
prompt_RAG = """
    You are a proficient python developer. Respond with the syntactically correct code for to the question below. Make sure you follow these rules:
    1. Use context to understand the APIs and how to use it & apply.
    2. Do not add license information to the output code.
    3. Do not include Colab code in the output.
    4. Ensure all the requirements in the question are met.

    Question:
    {question}

    Context:
    {context}

    Helpful Response :
    """

prompt_RAG_template = PromptTemplate(
    template=prompt_RAG, input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_llm(
    llm=code_llm,
    prompt=prompt_RAG_template,
    retriever=retriever,
    return_source_documents=True,
)

### 5. Try zero-shot prompt

In [32]:
response = code_llm.predict(text=user_question, max_output_tokens=2048, temperature=0.1)
print(response)

/root/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


```python
def predict_with_langchain_llms(prompt):
    """Predicts the next token using the langchain.llms interface with Vertex AI text-bison model.

    Args:
        prompt: The input text prompt.

    Returns:
        The predicted next token.
    """

    # Import the necessary libraries.
    import requests

    # Set the endpoint of the Vertex AI text-bison model.
    endpoint = "https://us-central1-aiplatform.googleapis.com/v1/projects/YOUR_PROJECT_ID/locations/us-central1/models/YOUR_MODEL_ID:predict"

    # Set the request body.
    body = {
        "instances": [
            {
                "text": prompt,
            }
        ]
    }

    # Make the request to the Vertex AI text-bison model.
    response = requests.post(endpoint, json=body)

    # Get the predicted next token from the response.
    predicted_token = response.json()["predictions"][0]["text"]

    # Return the predicted next token.
    return predicted_token
```


### 6. Run prompt using RAG Chain & compare results
To generate response we use code-bison however can also use code-gecko and codechat-bison

In [33]:
results = qa_chain({"query": user_question})
print(results["result"])

/root/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


```python
def predict_with_langchain_llms(prompt: str) -> str:
    """Predicts using langchain.llms interface with Vertex AI text-bison model.

    Args:
        prompt: The prompt to predict.

    Returns:
        The predicted response.
    """

    # Initialize the LLM model.
    llm = VertexAI(
        model_name="text-bison",
        max_output_tokens=256,
        temperature=0.1,
        top_p=0.8,
        top_k=40,
        verbose=True,
    )

    # Predict the response.
    response = llm.predict(text=prompt, max_output_tokens=2048, temperature=0.1)

    return response

```


### Let's try another prompt

In [34]:
user_question = "Create python function that takes text input and returns embeddings using LangChain with Vertex AI textembedding-gecko model"


response = code_llm.predict(text=user_question, max_output_tokens=2048, temperature=0.1)
print(response)

```python
def get_embeddings_langchain(text):
    """Gets embeddings for a given text using LangChain with Vertex AI textembedding-gecko model.

    Args:
        text: The text to get embeddings for.

    Returns:
        A list of embeddings for the text.
    """

    # Import the necessary libraries.
    import google.cloud.aiplatform as aiplatform

    # Get the model.
    model = aiplatform.gapic.ModelServiceClient.get_model(
        name="projects/YOUR_PROJECT_ID/locations/YOUR_LOCATION/models/YOUR_MODEL_ID"
    )

    # Get the model's input and output tensor names.
    input_tensor_name = model.input_tensor_names[0]
    output_tensor_name = model.output_tensor_names[0]

    # Create the input instance.
    input_instance = {"inputs": text}

    # Get the embeddings.
    response = aiplatform.gapic.PredictionServiceClient.predict(
        name=model.name, instances=[input_instance]
    )

    # Get the embeddings from the response.
    embeddings = response.predictions[0][output

In [35]:
results = qa_chain({"query": user_question})
print(results["result"])

```python
import vertexai
from vertexai.language_models import TextEmbeddingModel
from vertexai.generative_models import GenerativeModel, ChatSession
import os
import pandas as pd
import chromadb
! gsutil -m cp -r gs://github-repo/generative-ai/gemini/use-cases/rag/small-to-big-rag/onlineboutique-codefiles .
model = TextEmbeddingModel.from_pretrained(EMBEDDING_MODEL)


def get_text_embedding(doc) -> list:
    embeddings = model.get_embeddings([doc])
    if len(embeddings) > 1:
        raise ValueError("More than one embedding returned.")
    if len(embeddings) == 0:
        raise ValueError("No embedding returned.")
    return embeddings[0].values
vertexai.init(project=PROJECT_ID, location=REGION)
model = GenerativeModel(GENERATIVE_MODEL)
chat = model.start_chat()

```
